Use LRU eviction policy.# Typeahead 

## Functional 
- Suggests top 10 terms starting with whatever user has typed.

## Non-functional 
- Users see suggestions within 200ms.

## Design
- Trie structure.
- Store this data structure in memory. (Not in DB)
- Assume case insensitive

<img src="img/typeahead1.png" style="width:200px;height:500px;">

- For example, when user types "CAP", node "P" will show all suggestions.

### How to find top suggestions
- Store the number of searches in the last phrase.
- Parent nodes recursively call child nodes to get suggestions and counts.

### How to update trie
- Spread copies of tries on each server and update them offline. (While some nodes are serving the traffic)
- Also, remove inappropriate terms from the tries duing this time.
- Also, update frequency at each node.

### Client behavior
- Client should hit the server only when no key has been pressed for 50ms.
- When user is typing, client should cancel in-progress request.
- Client should wait until at least few characters are entered in the beginning.
- Client can store recent history of top suggestions.
- Establish connection between client and server as soon as browser opens up.
- Server should push some part of cache to CDNs.

## Capacity

### Assume
- 5B quries per day = 60K queries per second.
- From 5B queries, 100M are unique terms.
- Each query is 30 bytes.

### Storage
- Total storage needed: 100M * 30 bytes = 3GB

## Data partitioning
- Range based partitioning
    - Pick storage based on starting character of queries.
    - Results in uneven servers
    - There will be hot servers.
    - May not be able to fit the size of data into each server.
- Hash based partitioning
    - Each term gets a server number from hash function. 
    - Minimize hot servers.
    - Must query all servers and aggregate the result.

## Caching
- Cache servers in front of trie servers with frequenctly searched terms and their typeahead suggestions.
- App servers should check cache servers before hitting trie servers.